In [5]:
import librosa
import pretty_midi

# Load audio file using librosa
audio_file_path = '../audio/alto.wav'  # Replace with the path to your audio file
y, sr = librosa.load(audio_file_path)

# Extract pitch and beat information using librosa
pitches, magnitudes = librosa.core.piptrack(y=y, sr=sr)  # Extract pitch information
times = librosa.times_like(pitches)  # Get times corresponding to the pitch information

# Create a PrettyMIDI object
midi_data = pretty_midi.PrettyMIDI()

# Create an Instrument instance for the piano (program number 0)
piano_program = pretty_midi.instrument_name_to_program('Acoustic Grand Piano')
piano = pretty_midi.Instrument(program=piano_program)



# # Add notes to the Instrument instance based on the pitch information
# for pitch_list, magnitude, time in zip(pitches, magnitudes, times):
#     if np.any(np.array(magnitude) > 0):  # Consider only points with magnitude greater than 0
#         if isinstance(pitch_list, list):
#             for pitch in pitch_list:
#                 note_number = int(round(librosa.core.midi_to_note(pitch)))  # Convert MIDI pitch to note number
#                 velocity = int(np.max(magnitude) * 127)  # Scale magnitude to velocity (0-127 range)
#                 note = pretty_midi.Note(velocity=velocity, pitch=note_number, start=time, end=time + 0.1)  # Create note
#                 piano.notes.append(note)  # Add note to the Instrument instance
#         else:
#             note_number = int(round(librosa.core.midi_to_note(pitch_list)))  # Convert MIDI pitch to note number
#             velocity = int(np.max(magnitude) * 127)  # Scale magnitude to velocity (0-127 range)
#             note = pretty_midi.Note(velocity=velocity, pitch=note_number, start=time, end=time + 0.1)  # Create note
#             piano.notes.append(note)  # Add note to the Instrument instance



# Add notes to the Instrument instance based on the pitch information
for pitch, magnitude, time in zip(pitches, magnitudes, times):
    if magnitude > 0:  # Consider only points with magnitude greater than 0
        note_number = int(round(librosa.core.midi_to_note(pitch)))  # Convert MIDI pitch to note number
        velocity = int(magnitude * 127)  # Scale magnitude to velocity (0-127 range)
        note = pretty_midi.Note(velocity=velocity, pitch=note_number, start=time, end=time + 0.1)  # Create note
        piano.notes.append(note)  # Add note to the Instrument instance



# Add the piano Instrument to the MIDI data
midi_data.instruments.append(piano)

# Save the MIDI data to a file
midi_file_path = 'output.mid'  # Specify the path where you want to save the MIDI file
midi_data.write(midi_file_path)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [6]:
import librosa
import numpy as np
from scipy.signal import find_peaks
import mido
from mido import Message, MidiFile, MidiTrack

# Load the audio file
audio_file_path = '../audio/alto.wav'  # Replace with the path to your audio file
y, sr = librosa.load(audio_file_path)

# Function to extract fundamental frequency using PYIN pitch detection
def extract_fundamental_frequency(y, sr):
    # Use librosa's PYIN pitch detection algorithm
    pitches, magnitudes = librosa.core.piptrack(y=y, sr=sr)
    pitches = np.asarray(pitches)

    # Extract the fundamental frequency (lowest non-zero pitch)
    fundamental_frequency = pitches[np.any(pitches > 0, axis=1)].min(axis=1)

    # Remove zeros (unvoiced frames) from the fundamental frequency array
    fundamental_frequency = fundamental_frequency[fundamental_frequency > 0]

    return fundamental_frequency

# Extract fundamental frequency from the audio
fundamental_frequency = extract_fundamental_frequency(y, sr)

# Convert fundamental frequency to MIDI notes
def frequency_to_midi(frequency):
    return 69 + 12 * np.log2(frequency / 440.0)

midi_notes = list(map(int, np.round(frequency_to_midi(fundamental_frequency))))

# Create a MIDI file with the extracted notes
midi_file_path = 'output.mid'  # Specify the path where you want to save the MIDI file
midi = MidiFile()
track = MidiTrack()
midi.tracks.append(track)

for note in midi_notes:
    track.append(Message('note_on', note=note, velocity=64, time=100))  # Adjust time and velocity as needed
    track.append(Message('note_off', note=note, velocity=64, time=100))  # Adjust time as needed

# Save the MIDI file
midi.save(midi_file_path)
